In [176]:
# preprocessing
from sklearn import preprocessing

# nlp
import langdetect

import nltk
from nltk import sent_tokenize
from nltk import word_tokenize
from nltk.probability import FreqDist
from nltk.corpus import stopwords

# word cloud
from wordcloud import WordCloud
# visualization
import matplotlib.pyplot as plt

# algebra
import numpy as np

# dataframing
import pandas as pd

In [177]:
data = pd.read_csv("./data/youtube_dataset.csv")

## 0. About data

 - Data is about **YouTube comments**. It includes video that the comment was posted on, channel of the video, date, amount of likes the video received, etc.
 - In total, there is 7 features and 19300 instances.
 - Dataset was found and downloaded from https://www.kaggle.com/nipunarora8/most-liked-comments-on-youtube


## 1. Examine data

In [178]:
data.head()

,Video Name,Channel Name,Comment Id,User Name,Comment,Date,Likes
0,Luis Fonsi - Despacito ft. Daddy Yankee,LuisFonsiVEVO,UgwV0tapZzaFxdYm1z14AaABAg,GAMIT yuvraj,The people who liked this comment is officiall...,2020-08-10T20:00:43Z,98280
1,Luis Fonsi - Despacito ft. Daddy Yankee,LuisFonsiVEVO,UgyiccblP3A16aixpit4AaABAg,MrCrazy,"- Wait, it's 7B views\n- Always has been",2020-09-07T20:52:36Z,773
2,Luis Fonsi - Despacito ft. Daddy Yankee,LuisFonsiVEVO,Ugx7THW3EBNJlhJuwSB4AaABAg,60K Subs With 1 Video Challenge,*Teacher: What is the population of the Earth?...,2019-11-21T03:23:32Z,356081
3,Luis Fonsi - Despacito ft. Daddy Yankee,LuisFonsiVEVO,UgzXUsI6yrRjTKNASf94AaABAg,Srivathsa D,Let's be honest this wasn't your recommendatio...,2020-09-09T03:41:34Z,13
4,Luis Fonsi - Despacito ft. Daddy Yankee,LuisFonsiVEVO,UgxCu6mywg6uqjueHQ14AaABAg,1k Subscribers Challenge Without Any Video,Types Of People:\n10% Enjoying Song \n90% Chec...,2020-09-08T09:53:24Z,122


In [179]:
data.sample(frac = 1)

,Video Name,Channel Name,Comment Id,User Name,Comment,Date,Likes
13353,Queen – Bohemian Rhapsody (Official Video Rema...,Queen Official,UgyUBeObl3crxFeuLmt4AaABAg,Aashish,This song deserves much more VIEWS than some s...,2020-09-06T03:40:07Z,36
18151,twenty one pilots - Ride (Official Video),twenty one pilots,UgzEX8ha-XvEgrIWjdB4AaABAg,B e a t c l o u d,"When this song was trending, i had many good f...",2020-08-27T03:29:37Z,67
6618,Maluma - Felices los 4 (Official Video),MalumaVEVO,UgwheqHTlHAsugJOTXh4AaABAg,Ramingo,"""Felices los 4 iba a ser una alusión a las gue...",2020-09-05T18:47:26Z,2
10400,PSY - GENTLEMAN M/V,officialpsy,Ugz8EPCenvDBDCP8K2V4AaABAg,riyana khan,Psy need's a huge comeback,2020-06-04T18:09:44Z,24202
1082,Taylor Swift - Shake It Off,TaylorSwiftVEVO,UgyeqCpdzuCLgOo1Q6d4AaABAg,10K Subscribers Without Videos Challenge,"6 Years later, if you're watching this you're ...",2020-09-01T12:27:00Z,20
...,...,...,...,...,...,...,...
18836,"David Guetta - Play Hard ft. Ne-Yo, Akon (Offi...",David Guetta,Ugzln8CEUBMl8WYDvG14AaABAg,Alex Júlio de Moraes,Listening here the songs of all time now and a...,2020-08-13T19:46:27Z,53
9609,Una Lady Como Tú - MTZ Manuel Turizo | Video O...,MTZ Manuel Turizo,Ugz7PB019KnJuDNLF8x4AaABAg,efrain alvarez,Se les ase difícil aser una canción así?\n0% d...,2020-07-28T06:43:42Z,260
17032,Ariana Grande - Break Free ft. Zedd,ArianaGrandeVevo,UgxfEXEESDQHGmzmDvZ4AaABAg,The sisters channel,I’ma break free when quarantine is over.,2020-09-07T00:56:04Z,3
18219,Lukas Graham - 7 Years [OFFICIAL MUSIC VIDEO],Lukas Graham,Ugz_goqq6bcNxAZ-IFh4AaABAg,Austin G,I was just chilling in bed and this song just ...,2020-09-03T07:03:10Z,110


In [180]:
# most liked comments
data.sort_values("Likes",ascending=False)

,Video Name,Channel Name,Comment Id,User Name,Comment,Date,Likes
5201,Marshmello - Alone (Official Music Video),Marshmello,UgypW6-bk7J6JZJF1954AaABAg,Marshmello,"1,000,000,000 ✅",2018-10-15T19:33:03Z,540334
15805,Eminem - Rap God (Explicit) [Official Video],EminemVEVO,UgjtQw8E3CESxngCoAEC,Goldenightime,Eminem raps faster than my internet connection,2016-08-13T02:15:49Z,438973
5,Luis Fonsi - Despacito ft. Daddy Yankee,LuisFonsiVEVO,UgzryH8U6Dz_yBmIgEZ4AaABAg,LR ICED WOLF,3.2 Million comments if you find mine your a l...,2020-03-15T21:11:08Z,370547
2,Luis Fonsi - Despacito ft. Daddy Yankee,LuisFonsiVEVO,Ugx7THW3EBNJlhJuwSB4AaABAg,60K Subs With 1 Video Challenge,*Teacher: What is the population of the Earth?...,2019-11-21T03:23:32Z,356081
3120,Justin Bieber - Baby (Official Music Video) ft...,JustinBieberVEVO,Ugzv_LcinEv-IMMQZZN4AaABAg,Siddhant Dubey,There's 4.6M comments if you find mine then y...,2019-07-20T06:46:38Z,350917
...,...,...,...,...,...,...,...
11089,El Amante - Nicky Jam (Video Oficial) (Álbum ...,NickyJamTV,Ugyndg3MQJbNCdEGZHF4AaABAg,Josthin Cobeña,ME RIO DEL MIEDO,2020-06-28T18:34:13Z,0
11090,El Amante - Nicky Jam (Video Oficial) (Álbum ...,NickyJamTV,Ugz3Wy-qB08sAmy3zqV4AaABAg,Yisela Chaves,this song is beautiful,2020-06-23T22:36:37Z,0
4393,Shawn Mendes - Treat You Better,ShawnMendesVEVO,UgzScCcK6uBP92JF6yl4AaABAg,UwU,I make vid using this song and send this to my...,2020-09-08T12:21:43Z,0
11091,El Amante - Nicky Jam (Video Oficial) (Álbum ...,NickyJamTV,UgzZnAQdvpSS7knETiR4AaABAg,Hades Uwu,2017 el mejor año por ahora 🔥,2020-08-18T19:26:01Z,0


## 2. Preprocessing

### 1. Duplicates

In [181]:
# check for duplicates
data['Comment Id'].value_counts()[:10]

UgwynZ-b2duW4Q4HHF14AaABAg    2
UgytzfxBW8UXeQ6Xg1N4AaABAg    2
UgxNLWelD3mrLsitCbF4AaABAg    2
UgyupsCD7WYForow3jx4AaABAg    2
UgxXSdJ3i-S_uko7aw54AaABAg    2
Ugz2b7D-9NC5fqnrF1J4AaABAg    2
Ugx3RbZU2_iiGYVVMDd4AaABAg    2
Ugy_veYE8WmD5JSBz0h4AaABAg    2
UgzuhwJEjuC-b3lQ8vF4AaABAg    2
UgwIPbGJ8YdesLP9Zop4AaABAg    2
Name: Comment Id, dtype: int64

In [182]:
# remove duplicates
data.drop_duplicates(['Comment Id'], keep='first', inplace=True)

In [183]:
# check result
data['Comment Id'].value_counts()[:10]

UgzgRjw66DMlKB5Z7Tx4AaABAg    1
UgxH7dS7_V5c0p7K1R54AaABAg    1
UgyWHcjM9btt0C2Q3554AaABAg    1
UgwE8-k4l09SNpqjxNF4AaABAg    1
Ugyj6bKQhU0jFx1unr94AaABAg    1
UgwBjNPbKSGRfORtb9l4AaABAg    1
UgysEkV_HpgDnV2J93d4AaABAg    1
Ugx7Jd7vg1QrKJjeaht4AaABAg    1
Ugxy500YO1hHfpEb8l94AaABAg    1
UgxleCV8lTmyie5t8Yl4AaABAg    1
Name: Comment Id, dtype: int64

### 2. Standardization

Let's create a new feature that shows the like count as **like score**. The scale in likes is quite large. 

Let's try different standardization methods:

In [184]:
# 1. Z-score standardization: x - µ / sigma
y = data[['Likes']]
like_score_z_score = preprocessing.StandardScaler().fit_transform(y)

In [185]:
# Examine results
print(like_score_z_score)
print(max(like_score_z_score), min(like_score_z_score))

[[ 8.13349356]
 [-0.1156597 ]
 [29.94361958]
 ...
 [-0.18063298]
 [-0.18071758]
 [-0.18080218]]
[45.53153875] [-0.18105598]


In [186]:
# 2. Logarithmic standardization, apply logarithm for every value.
y_not_zero = data['Likes'].apply(lambda x: x + 1e-10)  # so that x > 0 for all x.

In [187]:
like_score_log = np.log(y_not_zero)

In [188]:
print(like_score_log)
print(max(like_score_log), min(like_score_log))

0        11.495576
1         6.650279
2        12.782914
3         2.564949
4         4.804021
           ...    
19295     8.534444
19296     1.791759
19297     1.609438
19298     1.386294
19299     1.098612
Name: Likes, Length: 19121, dtype: float64
13.199942745855235 -23.025850929940457


In [189]:
# 3. Min-max-scaling, scales values between 0 and 1.
like_score_minmax = preprocessing.MinMaxScaler().fit_transform(y)

In [190]:
print(like_score_minmax)
print(max(like_score_minmax), min(like_score_minmax))

[[1.81887499e-01]
 [1.43059663e-03]
 [6.59001655e-01]
 ...
 [9.25353578e-06]
 [7.40282862e-06]
 [5.55212147e-06]]
[1.] [0.]


Let's use these three "like scores" as ground truths and examine on the way how does the standardization affect our results.

### 3. Language detection

A problem: The comments are written in multiple different languages. Later, when extracting patterns from comments, multiple languages might bring problems. For example Japanese and Korean languages don't write spaces. Detecting words is hence challenging with the same logic than with for example English. 

So, let's try to detect the language of the comments with tools available, and examine the results.

Let's try library called **langdetect**, which has a support for 55 languages. It is not perfect, and especially short/single word sentences causes trouble for it. We don't have ground truths for languages, but let's take a random sample of 100 and examine manually how the library succeeds.

In [191]:
comments = data.sample(n=100, random_state=1)['Comment'].values
for c in comments:
    print(c)
    try:
        print(langdetect.detect(c))
    except:
        pass

Mi novia me dejo por que decía que me gusta mucho el reggaetón 
Pero volverá, para seducirme otra vez, besarme otra vez volverme a poner el mundo al revés. 🤷🏿‍♂️
es
Yes Mari and Keone!!!!!!!!!
en
jajaja ches mancos yo desde chiquita yo sabía que la morena era la Taylor
es
I've counted too many stars, now I'm an astronomer.
en
Found out I’m twice this guy’s age but I still can’t help but wanna play it
en
WALKERS TOGETHER AND FOREVER 😎😎😎TAKE ME THROUGH THE NIGHT FALL INTO THE DARKSIDE WE DON'T NEED THE LIGHT WERE LIVE ON THE DARK SIDE
en
Do you recall, not long ago
We would walk on the sidewalk
Innocent, remember?
All we did was care for each other

But the night was warm
We were bold and young
All around the wind blows
We would only hold on to let go

Blow a kiss, fire a gun
We need someone to lean on
Blow a kiss, fire a gun
All we need is somebody to lean on

Blow a kiss, fire a gun
We need someone to lean on
Blow a kiss, fire a gun
All we need is somebody to lean on

What will we do w

We can see that from this sample that
1. Most of the comments are in English.
2. Further, most of the languages are correctly detected (I spotted three mistakes).

Dominance of English in this sample means that we cannot examine well enough how does the detection work for other languages (except maybe Spanish and Portugease). To play it safer, let's use only the comments which are detected as English.

In [255]:
def detect_lang(comment: str):
    """
    Embed try / except syntax for detecting languages.
    """
    
    try:
        return langdetect.detect(comment)
    except:
        return comment

# apply detection for each comment
lang = data['Comment'].apply(detect_lang)

In [266]:
print(lang)
print("Number of English comments:", len(lang[lang=='en']))
print("% of total number of comments: ", (len(lang[lang=='en']) / len(data)) * 100)

0        en
1        en
2        en
3        en
4        en
         ..
19295    en
19296    en
19297    en
19298    en
19299    es
Name: Comment, Length: 19121, dtype: object
Number of English comments: 13188
% of total number of comments:  68.97128811254642


## 3. NLP

Let's take a random comment and test NLP methods on it

In [192]:
comment = data.sample(n=100)['Comment'].values
print(comment)

['Dicaprio: Meet my son Monke\nColdplay:'
 "Can today's whole pop industry combined make up for this?"
 'Se hace la que no me conoce dijo el Bugs Bunny :v'
 'I listened to this song again during quarantine, this nostalgia brings calm.'
 "I'll tell my kids this is first Rain On Me."
 'I just lost my dog of 4 years 😭😭😔😨😨' 'Timeless definition: THIS SONG'
 'Can I be blamed for being terrified of her when I was like 5 and this video was released?'
 'The fact is this comment section is still active after 10 years lit'
 "I can't be the only one who came to test my speakers."
 'The cage symbolizes the internet and its saying there are predators out there and little girls shouldn’t be on the web misleading these predators 😂 😝 😆'
 '🎤🎤😍😍😍😍😍🎤🎤\n🎤🎤🎤🎤😍🎤🎤🎤🎤\n🎤🎤🎤🎤😍🎤🎤🎤🎤\n🎤🎤🎤🎤😍🎤🎤🎤🎤\n🎤🎤🎤🎤😍🎤🎤🎤🎤\n🎤🎤🎤🎤🎤🎤🎤🎤🎤\n🎤🎤😍🎤🎤🎤😍🎤🎤\n🎤🎤😍🎤🎤🎤😍🎤🎤\n🎤🎤😍🎤😍🎤😍🎤🎤\n🎤🎤😍🎤😍🎤😍🎤🎤\n🎤🎤😍😍😍😍😍🎤🎤\n🎤🎤🎤🎤🎤🎤🎤🎤🎤\n🎤🎤😍😍😍😍😍🎤🎤\n🎤🎤😍🎤🎤🎤🎤🎤🎤\n🎤🎤😍😍😍😍😍🎤🎤\n🎤🎤😍🎤🎤🎤🎤🎤🎤\n🎤🎤😍😍😍😍😍🎤🎤\n🎤🎤🎤🎤🎤🎤🎤🎤🎤\n🎤🎤😍🎤🎤🎤😍🎤🎤\n🎤🎤😍😍🎤🎤😍🎤🎤\n🎤🎤😍🎤😍🎤😍🎤🎤\n🎤🎤😍🎤🎤😍😍🎤🎤\n🎤🎤😍🎤🎤🎤😍🎤🎤\n🎤🎤🎤🎤🎤🎤🎤🎤🎤\n🎤🎤😍

In [193]:
sentences = sent_tokenize(comment)

TypeError: cannot use a string pattern on a bytes-like object

In [ ]:
sentences

In [ ]:
# tokenize the text with words
words = word_tokenize(comment)
len(words)

In [ ]:
words

### 1. Frequent words

In [ ]:
# frequent words
fdist = FreqDist(words)
fdist.most_common(10)

In [ ]:
fdist.plot(10)

### 2. Remove punctuation marks

In [ ]:
words_no_punc = [w.lower() for w in words if w.isalpha()]
words_no_punc

In [ ]:
len(words_no_punc)

Punctuation marks removed.

In [ ]:
fdist = FreqDist(words_no_punc)
fdist.most_common(10)

In [ ]:
fdist.plot(10)

### 3. Remove stopwords

In [ ]:
stopwords = stopwords.words("spanish")
print(stopwords)

In [ ]:
words_no_stopwords = [w for w in words_no_punc if w not in stopwords]
print(words_no_stopwords)
print(len(words_no_stopwords))

### 4. Final frequency distribution

In [ ]:
fdist = FreqDist(words_no_stopwords)
fdist.most_common(10)

In [ ]:
fdist.plot(10)
print(str(words_no_stopwords))

In [ ]:
wc = WordCloud().generate(comment)
plt.figure(figsize = (12, 12))
plt.imshow(wc)

In [ ]:
 from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vectorizer = CountVectorizer() #Convert a collection of text documents to a matrix of token counts
X = vectorizer.fit_transform(comment)

In [ ]:
feature_names = vectorizer.get_feature_names()
feature_names

In [ ]:
data = X.toarray()   
data